In [18]:
import cv2
import os
import numpy as np
from itertools import combinations
import pygetwindow as gw
import mss
import time

WINDOW_TITLE = "World Series of Poker - Google Chrome"
TEMPLATE_DIRS = {
    'rank':    'rank',
    'suit':    'suit',
    'rank-h1': 'rank-h1',
    'suit-h1': 'suit-h1',
    'rank-h2': 'rank-h2',
    'suit-h2': 'suit-h2',
}
REGIONS = {
    'f1-rank':   (631, 526,  79,  99),
    'f1-suit':   (631, 620,  76,  88),
    'f2-rank':   (803, 526,  79,  99),
    'f2-suit':   (803, 620,  76,  88),
    'f3-rank':   (980, 526,  79,  99),
    'f3-suit':   (980, 620,  76,  88),
    'f4-rank':   (1155,526,  79,  99),
    'f4-suit':   (1155,620,  76,  88),
    'f5-rank':   (1330,526,  79,  99),
    'f5-suit':   (1330,620,  76,  88),
    'h1-1-rank': (625, 806,  86,  96),
    'h1-1-suit': (639, 905,  71,  73),
    'h1-2-rank': (723, 798,  79,  99),
    'h1-2-suit': (719, 899,  67,  71),
    'h2-1-rank': (1245,793,  81, 115),
    'h2-1-suit': (1245,900,  71,  73),
    'h2-2-rank': (1336,798,  79,  99),
    'h2-2-suit': (1321,890,  72,  79),
}
RANK_MAP = {'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,
            't':10,'j':11,'q':12,'k':13,'a':14}

def evaluate_5cards(cards):
    ranks = sorted([RANK_MAP[r] for r, s in cards], reverse=True)
    suits = [s for r, s in cards]
    counts = {r: ranks.count(r) for r in set(ranks)}
    sorted_counts = sorted(counts.items(), key=lambda x: (-x[1], -x[0]))
    flush = len(set(suits)) == 1
    unique = sorted(set(ranks), reverse=True)
    straight = False
    high_st = None
    if len(unique) == 5:
        if unique[0] - unique[-1] == 4:
            straight, high_st = True, unique[0]
        if unique == [14, 5, 4, 3, 2]:
            straight, high_st = True, 5

    if straight and flush:
        return (9, high_st)
    if sorted_counts[0][1] == 4:
        four = sorted_counts[0][0]
        kicker = max(r for r in ranks if r != four)
        return (8, four, kicker)
    if sorted_counts[0][1] == 3 and sorted_counts[1][1] == 2:
        return (7, sorted_counts[0][0], sorted_counts[1][0])
    if flush:
        return (6, *ranks)
    if straight:
        return (5, high_st)
    if sorted_counts[0][1] == 3:
        trip = sorted_counts[0][0]
        kickers = sorted([r for r in ranks if r != trip], reverse=True)
        return (4, trip, *kickers)
    if sorted_counts[0][1] == 2 and sorted_counts[1][1] == 2:
        pair1, pair2 = sorted_counts[0][0], sorted_counts[1][0]
        kicker = max(r for r in ranks if r not in (pair1, pair2))
        return (3, max(pair1, pair2), min(pair1, pair2), kicker)
    if sorted_counts[0][1] == 2:
        pair = sorted_counts[0][0]
        kickers = sorted([r for r in ranks if r != pair], reverse=True)
        return (2, pair, *kickers)
    return (1, *ranks)

def best_hand_7(cards7):
    best = None
    for combo in combinations(cards7, 5):
        score = evaluate_5cards(combo)
        if best is None or score > best:
            best = score
    return best

def load_templates(path):
    templates = []
    for fname in os.listdir(path):
        tpl = cv2.imread(os.path.join(path, fname), cv2.IMREAD_GRAYSCALE)
        if tpl is not None:
            name = os.path.splitext(fname)[0]
            templates.append((name.lower(), tpl))
    return templates

def match_best(crop, templates):
    best_name = None
    best_val = -1
    for name, tpl in templates:
        h, w = tpl.shape
        if crop.shape[0] >= h and crop.shape[1] >= w:
            res = cv2.matchTemplate(crop, tpl, cv2.TM_CCOEFF_NORMED)
            _, maxv, _, _ = cv2.minMaxLoc(res)
            if maxv > best_val:
                best_val, best_name = maxv, name
    return best_name

def find_window_rect(title):
    wins = gw.getWindowsWithTitle(title)
    if wins:
        win = wins[0]
        return (win.left, win.top, win.width, win.height, win)
    else:
        return None

def is_window_focused(target_win):
    try:
        focused = gw.getActiveWindow()
        return focused == target_win
    except Exception:
        return False

def render_results_img(winner_text, hand1_score, hand2_score):
    # Blank white image
    img = np.ones((200, 400, 3), dtype=np.uint8) * 255
    font = cv2.FONT_HERSHEY_SIMPLEX

    # Prepare lines
    lines = [
        winner_text,
        f'Hand1: {hand1_score}',
        f'Hand2: {hand2_score}'
    ]
    y = 60
    for i, text in enumerate(lines):
        size = cv2.getTextSize(text, font, 0.8, 2)[0]
        x = (img.shape[1] - size[0]) // 2
        cv2.putText(img, text, (x, y), font, 0.8, (0, 0, 0), 2, cv2.LINE_AA)
        y += size[1] + 25
    return img

def render_status_img(message):
    img = np.ones((200, 400, 3), dtype=np.uint8) * 240
    font = cv2.FONT_HERSHEY_SIMPLEX
    size = cv2.getTextSize(message, font, 0.8, 2)[0]
    x = (img.shape[1] - size[0]) // 2
    y = img.shape[0] // 2 + size[1] // 2
    cv2.putText(img, message, (x, y), font, 0.8, (20, 20, 20), 2, cv2.LINE_AA)
    return img

def main():
    print("Loading templates...")
    tpl_sets = {k: load_templates(v) for k, v in TEMPLATE_DIRS.items()}

    print(f"Looking for window titled: '{WINDOW_TITLE}'")
    win_info = None
    target_win = None
    while True:
        found = find_window_rect(WINDOW_TITLE)
        if found:
            left, top, width, height, target_win = found
            print("Found window!")
            break
        print("Window not found, retrying in 1 second...")
        status_img = render_status_img("Window not found.\nWaiting...")
        cv2.imshow('Poker Results', status_img)
        if cv2.waitKey(1000) == 27:
            cv2.destroyAllWindows()
            return

    sct = mss.mss()
    monitor = {"left": left, "top": top, "width": width, "height": height}

    print("Ready! Starting live detection.")
    while True:
        # Check window presence
        found = find_window_rect(WINDOW_TITLE)
        if not found:
            print("Window lost! Retrying...")
            status_img = render_status_img("Window lost!\nWaiting...")
            cv2.imshow('Poker Results', status_img)
            if cv2.waitKey(1000) == 27:
                break
            continue
        _, _, _, _, win = found

        # Check focus
        if not is_window_focused(win):
            print("Window not focused! Waiting...")
            status_img = render_status_img("Window not focused!")
            cv2.imshow('Poker Results', status_img)
            if cv2.waitKey(500) == 27:
                break
            continue

        # Capture and process
        sct_img = sct.grab(monitor)
        frame = np.array(sct_img)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGRA2BGR)
        img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        results = {}
        for lbl, (x, y, w, h) in REGIONS.items():
            cid, kind = lbl.rsplit('-', 1)
            crop = img_gray[y:y+h, x:x+w]
            if kind == 'rank':
                key = 'rank-h1' if cid == 'h1-1' else 'rank-h2' if cid == 'h2-2' else 'rank'
            else:
                key = 'suit-h1' if cid == 'h1-1' else 'suit-h2' if cid == 'h2-2' else 'suit'
            match = match_best(crop, tpl_sets[key])
            results.setdefault(cid, {})[kind] = match

        flop = [(results[f]['rank'], results[f]['suit']) for f in ['f1','f2','f3','f4','f5']]
        h1 = [(results['h1-1']['rank'], results['h1-1']['suit']),
              (results['h1-2']['rank'], results['h1-2']['suit'])]
        h2 = [(results['h2-1']['rank'], results['h2-1']['suit']),
              (results['h2-2']['rank'], results['h2-2']['suit'])]

        s1 = best_hand_7(flop + h1)
        s2 = best_hand_7(flop + h2)
        if s1 > s2:
            winner_text = "Wybierz hand1"
        elif s2 > s1:
            winner_text = "Wybierz hand2"
        else:
            winner_text = "Remis"

        # Prepare result display
        result_img = render_results_img(winner_text, s1, s2)
        cv2.imshow('Poker Results', result_img)

        key = cv2.waitKey(30)
        if key == 27:
            break

    cv2.destroyAllWindows()

main()


Loading templates...
Looking for window titled: 'World Series of Poker - Google Chrome'
Found window!
Ready! Starting live detection.
Window not focused! Waiting...
Window not focused! Waiting...
Window not focused! Waiting...
Window not focused! Waiting...
Window not focused! Waiting...
Window not focused! Waiting...


In [19]:
import cv2
import os
import numpy as np
from itertools import combinations
import pygetwindow as gw
import mss
import time
from datetime import datetime

WINDOW_TITLE = "World Series of Poker - Google Chrome"
TEMPLATE_DIRS = {
    'rank':    'rank',
    'suit':    'suit',
    'rank-h1': 'rank-h1',
    'suit-h1': 'suit-h1',
    'rank-h2': 'rank-h2',
    'suit-h2': 'suit-h2',
}
REGIONS = {
    'f1-rank':   (631, 526,  79,  99),
    'f1-suit':   (631, 620,  76,  88),
    'f2-rank':   (803, 526,  79,  99),
    'f2-suit':   (803, 620,  76,  88),
    'f3-rank':   (980, 526,  79,  99),
    'f3-suit':   (980, 620,  76,  88),
    'f4-rank':   (1155,526,  79,  99),
    'f4-suit':   (1155,620,  76,  88),
    'f5-rank':   (1330,526,  79,  99),
    'f5-suit':   (1330,620,  76,  88),
    'h1-1-rank': (625, 806,  86,  96),
    'h1-1-suit': (639, 905,  71,  73),
    'h1-2-rank': (723, 798,  79,  99),
    'h1-2-suit': (719, 899,  67,  71),
    'h2-1-rank': (1245,793,  81, 115),
    'h2-1-suit': (1245,900,  71,  73),
    'h2-2-rank': (1336,798,  79,  99),
    'h2-2-suit': (1321,890,  72,  79),
}
RANK_MAP = {'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,
            't':10,'j':11,'q':12,'k':13,'a':14}

if not os.path.exists('sc'):
    os.makedirs('sc')

def evaluate_5cards(cards):
    ranks = sorted([RANK_MAP[r] for r, s in cards], reverse=True)
    suits = [s for r, s in cards]
    counts = {r: ranks.count(r) for r in set(ranks)}
    sorted_counts = sorted(counts.items(), key=lambda x: (-x[1], -x[0]))
    flush = len(set(suits)) == 1
    unique = sorted(set(ranks), reverse=True)
    straight = False
    high_st = None
    if len(unique) == 5:
        if unique[0] - unique[-1] == 4:
            straight, high_st = True, unique[0]
        if unique == [14, 5, 4, 3, 2]:
            straight, high_st = True, 5

    if straight and flush:
        return (9, high_st)
    if sorted_counts[0][1] == 4:
        four = sorted_counts[0][0]
        kicker = max(r for r in ranks if r != four)
        return (8, four, kicker)
    if sorted_counts[0][1] == 3 and sorted_counts[1][1] == 2:
        return (7, sorted_counts[0][0], sorted_counts[1][0])
    if flush:
        return (6, *ranks)
    if straight:
        return (5, high_st)
    if sorted_counts[0][1] == 3:
        trip = sorted_counts[0][0]
        kickers = sorted([r for r in ranks if r != trip], reverse=True)
        return (4, trip, *kickers)
    if sorted_counts[0][1] == 2 and sorted_counts[1][1] == 2:
        pair1, pair2 = sorted_counts[0][0], sorted_counts[1][0]
        kicker = max(r for r in ranks if r not in (pair1, pair2))
        return (3, max(pair1, pair2), min(pair1, pair2), kicker)
    if sorted_counts[0][1] == 2:
        pair = sorted_counts[0][0]
        kickers = sorted([r for r in ranks if r != pair], reverse=True)
        return (2, pair, *kickers)
    return (1, *ranks)

def best_hand_7(cards7):
    best = None
    for combo in combinations(cards7, 5):
        score = evaluate_5cards(combo)
        if best is None or score > best:
            best = score
    return best

def load_templates(path):
    templates = []
    for fname in os.listdir(path):
        tpl = cv2.imread(os.path.join(path, fname), cv2.IMREAD_GRAYSCALE)
        if tpl is not None:
            name = os.path.splitext(fname)[0]
            templates.append((name.lower(), tpl))
    return templates

def match_best(crop, templates):
    best_name = None
    best_val = -1
    for name, tpl in templates:
        h, w = tpl.shape
        if crop.shape[0] >= h and crop.shape[1] >= w:
            res = cv2.matchTemplate(crop, tpl, cv2.TM_CCOEFF_NORMED)
            _, maxv, _, _ = cv2.minMaxLoc(res)
            if maxv > best_val:
                best_val, best_name = maxv, name
    return best_name

def find_window_rect(title):
    wins = gw.getWindowsWithTitle(title)
    if wins:
        win = wins[0]
        return (win.left, win.top, win.width, win.height, win)
    else:
        return None

def is_window_focused(target_win):
    try:
        focused = gw.getActiveWindow()
        return focused == target_win
    except Exception:
        return False

def render_results_img(winner_text, hand1_score, hand2_score):
    img = np.ones((200, 400, 3), dtype=np.uint8) * 255
    font = cv2.FONT_HERSHEY_SIMPLEX
    lines = [winner_text, f'Hand1: {hand1_score}', f'Hand2: {hand2_score}']
    y = 60
    for i, text in enumerate(lines):
        size = cv2.getTextSize(text, font, 0.8, 2)[0]
        x = (img.shape[1] - size[0]) // 2
        cv2.putText(img, text, (x, y), font, 0.8, (0, 0, 0), 2, cv2.LINE_AA)
        y += size[1] + 25
    return img

def render_status_img(message):
    img = np.ones((200, 400, 3), dtype=np.uint8) * 240
    font = cv2.FONT_HERSHEY_SIMPLEX
    size = cv2.getTextSize(message, font, 0.8, 2)[0]
    x = (img.shape[1] - size[0]) // 2
    y = img.shape[0] // 2 + size[1] // 2
    cv2.putText(img, message, (x, y), font, 0.8, (20, 20, 20), 2, cv2.LINE_AA)
    return img

def main():
    print("Loading templates...")
    tpl_sets = {k: load_templates(v) for k, v in TEMPLATE_DIRS.items()}

    print(f"Looking for window titled: '{WINDOW_TITLE}'")
    win_info = None
    target_win = None
    while True:
        found = find_window_rect(WINDOW_TITLE)
        if found:
            left, top, width, height, target_win = found
            print("Found window!")
            break
        print("Window not found, retrying in 1 second...")
        status_img = render_status_img("Window not found.\nWaiting...")
        cv2.imshow('Poker Results', status_img)
        if cv2.waitKey(1000) == 27:
            cv2.destroyAllWindows()
            return

    sct = mss.mss()
    monitor = {"left": left, "top": top, "width": width, "height": height}

    last_save = 0  # For 5-second screenshot interval

    print("Ready! Starting live detection.")
    while True:
        found = find_window_rect(WINDOW_TITLE)
        if not found:
            print("Window lost! Retrying...")
            status_img = render_status_img("Window lost!\nWaiting...")
            cv2.imshow('Poker Results', status_img)
            if cv2.waitKey(1000) == 27:
                break
            continue
        _, _, _, _, win = found

        if not is_window_focused(win):
            print("Window not focused! Waiting...")
            status_img = render_status_img("Window not focused!")
            cv2.imshow('Poker Results', status_img)
            if cv2.waitKey(500) == 27:
                break
            continue

        sct_img = sct.grab(monitor)
        frame = np.array(sct_img)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGRA2BGR)

        # --- Crop 9px from left before any further use ---
        frame = frame[:, 9:, :]

        # --- Save screenshot every 5 seconds ---
        current_time = time.time()
        if current_time - last_save >= 5:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
            cv2.imwrite(f"sc/{timestamp}.png", frame)
            last_save = current_time

        img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        results = {}
        for lbl, (x, y, w, h) in REGIONS.items():
            cid, kind = lbl.rsplit('-', 1)
            crop = img_gray[y:y+h, x:x+w]
            if kind == 'rank':
                key = 'rank-h1' if cid == 'h1-1' else 'rank-h2' if cid == 'h2-2' else 'rank'
            else:
                key = 'suit-h1' if cid == 'h1-1' else 'suit-h2' if cid == 'h2-2' else 'suit'
            match = match_best(crop, tpl_sets[key])
            results.setdefault(cid, {})[kind] = match

        flop = [(results[f]['rank'], results[f]['suit']) for f in ['f1','f2','f3','f4','f5']]
        h1 = [(results['h1-1']['rank'], results['h1-1']['suit']),
              (results['h1-2']['rank'], results['h1-2']['suit'])]
        h2 = [(results['h2-1']['rank'], results['h2-1']['suit']),
              (results['h2-2']['rank'], results['h2-2']['suit'])]

        s1 = best_hand_7(flop + h1)
        s2 = best_hand_7(flop + h2)
        if s1 > s2:
            winner_text = "Wybierz hand1"
        elif s2 > s1:
            winner_text = "Wybierz hand2"
        else:
            winner_text = "Remis"

        result_img = render_results_img(winner_text, s1, s2)
        cv2.imshow('Poker Results', result_img)

        key = cv2.waitKey(30)
        if key == 27:
            break

    cv2.destroyAllWindows()

main()


Loading templates...
Looking for window titled: 'World Series of Poker - Google Chrome'
Found window!
Ready! Starting live detection.
Window not focused! Waiting...
Window not focused! Waiting...
Window not focused! Waiting...
Window not focused! Waiting...
Window not focused! Waiting...
Window not focused! Waiting...
Window not focused! Waiting...
Window not focused! Waiting...


In [25]:
import cv2

# Wczytaj obraz
image = cv2.imread('bth.png')

# Zaznacz ROI myszką – otworzy się interfejs graficzny
roi = cv2.selectROI("Zaznacz obszar", image, showCrosshair=True)

# Zamknij okno
cv2.destroyAllWindows()

# Wynik: (x, y, width, height)
print("ROI:", roi)
x, y, w, h = roi
region = image[y:y+h, x:x+w]


ROI: (642, 717, 145, 41)


In [4]:
import cv2
import numpy as np
from PIL import Image
import os
import re

main_img_path = 'bth.png'
template_folders = {
    'card1': 'templates/card1',
    'card2': 'templates/card2',
    'card3': 'templates/card3',
}
card_regions = {
    'card1': (688, 612, 75, 94),
    'card2': (728, 586, 75, 94),
    'card3': (772, 586, 78, 94),
}

main_img = cv2.imread(main_img_path)
if main_img is None:
    raise FileNotFoundError("Could not load bth.png")

# --- Sprawdzenie koloru piksela (684, 743) ---
pixel_coords = (684, 743)
expected_bgr = np.array([255, 254, 254], dtype=np.uint8)
h, w = main_img.shape[:2]
if 0 <= pixel_coords[0] < w and 0 <= pixel_coords[1] < h:
    pixel_bgr = main_img[pixel_coords[1], pixel_coords[0]]
    pixel_rgb = pixel_bgr[::-1]
    print(f"Piksel {pixel_coords} — BGR: {tuple(pixel_bgr)}, RGB: {tuple(pixel_rgb)}")
    if np.all(pixel_bgr == expected_bgr):
        print("Kolor się zgadza (BEZ tolerancji).")
    else:
        print(f"Kolor NIEZGODNY z oczekiwanym BGR: {tuple(expected_bgr)}")
else:
    print("Podany piksel jest poza zakresem obrazu.")

# --- Dopasowywanie templatek ---
def extract_number(filename):
    # Wyciąga pierwszą liczbę z nazwy pliku, np. '13.png' -> 13
    match = re.search(r'\d+', filename)
    return int(match.group()) if match else None

for card_key, (x, y, w, h) in card_regions.items():
    region_img = main_img[y:y+h, x:x+w]
    region_gray = cv2.cvtColor(region_img, cv2.COLOR_BGR2GRAY)
    _, region_bin = cv2.threshold(region_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    best_score = -1
    best_template_name = None

    template_dir = template_folders[card_key]
    for filename in os.listdir(template_dir):
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue
        template_path = os.path.join(template_dir, filename)
        template_gray = np.array(Image.open(template_path).convert('L'))
        _, template_bin = cv2.threshold(template_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        t_h, t_w = template_bin.shape[:2]
        r_h, r_w = region_bin.shape[:2]
        if t_h > r_h or t_w > r_w:
            continue

        res = cv2.matchTemplate(region_bin, template_bin, cv2.TM_CCOEFF_NORMED)
        max_val = cv2.minMaxLoc(res)[1]
        if max_val > best_score:
            best_score = max_val
            best_template_name = filename

    # Zwraca numer (int) zamiast nazwy pliku!
    card_number = extract_number(best_template_name) if best_template_name else None
    print(f"{card_key}: {card_number} (score {best_score:.4f})")


Piksel (684, 743) — BGR: (np.uint8(255), np.uint8(254), np.uint8(254)), RGB: (np.uint8(254), np.uint8(254), np.uint8(255))
Kolor się zgadza (BEZ tolerancji).
card1: 4 (score 0.9224)
card2: 8 (score 0.7786)
card3: 13 (score 0.8208)


In [1]:
import cv2
import numpy as np
from PIL import Image
import os
import re
import pygetwindow as gw
import pyautogui
import time
import threading
import tkinter as tk

template_folders = {
    'card1': 'templates/card1',
    'card2': 'templates/card2',
    'card3': 'templates/card3',
}
card_regions = {
    'card1': (688, 612, 75, 94),
    'card2': (728, 586, 75, 94),
    'card3': (772, 586, 78, 94),
}
window_title = "World Series of Poker - Google Chrome"
pixel_coords = (684, 743)
expected_bgr = np.array([255, 254, 254], dtype=np.uint8)
flush_coords = (633, 465)
flush_expect_bgr = np.array([15, 3, 69], dtype=np.uint8)

def extract_number(filename):
    match = re.search(r'\d+', filename)
    return int(match.group()) if match else None

def get_window_screenshot(title):
    windows = gw.getWindowsWithTitle(title)
    for w in windows:
        if w.isActive and w.width > 0 and w.height > 0:
            bbox = (w.left, w.top, w.left + w.width, w.top + w.height)
            img = pyautogui.screenshot(region=bbox)
            screenshot = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
            screenshot = screenshot[:, 9:, :]
            return screenshot, w.left, w.top
    return None, None, None

def recognize_cards(main_img):
    results = []
    for card_key, (x, y, w, h) in card_regions.items():
        region_img = main_img[y:y+h, x:x+w]
        region_gray = cv2.cvtColor(region_img, cv2.COLOR_BGR2GRAY)
        _, region_bin = cv2.threshold(region_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        best_score = -1
        best_template_name = None
        template_dir = template_folders[card_key]
        for filename in os.listdir(template_dir):
            if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue
            template_path = os.path.join(template_dir, filename)
            template_gray = np.array(Image.open(template_path).convert('L'))
            _, template_bin = cv2.threshold(template_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            t_h, t_w = template_bin.shape[:2]
            r_h, r_w = region_bin.shape[:2]
            if t_h > r_h or t_w > r_w:
                continue
            res = cv2.matchTemplate(region_bin, template_bin, cv2.TM_CCOEFF_NORMED)
            max_val = cv2.minMaxLoc(res)[1]
            if max_val > best_score:
                best_score = max_val
                best_template_name = filename
        card_number = extract_number(best_template_name) if best_template_name else None
        results.append((card_key, card_number, best_score))
    return results

def calculate_decision(card_numbers):
    cards_sorted = sorted(card_numbers)
    # 1. Przynajmniej jedna karta >= 13
    if any(card >= 13 for card in cards_sorted):
        return "Bet"
    # 2. Sekwens (np. 4,5,6)
    if all(cards_sorted[i] + 1 == cards_sorted[i+1] for i in range(2)):
        return "Bet"
    # 3. Specjalny przypadek: dokładnie (2,3,14) (po sortowaniu)
    if tuple(cards_sorted) == (2, 3, 14):
        return "Bet"
    # 4. Dwie takie same (para)
    if cards_sorted[0] == cards_sorted[1] or cards_sorted[1] == cards_sorted[2] or cards_sorted[0] == cards_sorted[2]:
        return "Bet"
    # 5. Trzy takie same (trójka)
    if cards_sorted[0] == cards_sorted[1] == cards_sorted[2]:
        return "Bet"
    return "Fold"

class ResultWindow:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Rozpoznanie kart WSOP")
        self.label = tk.Label(self.root, text="...", font=("Arial", 16, "bold"), justify='left')
        self.label.pack(padx=16, pady=18)
        self.root.geometry("200x70+30+30")
        self.root.resizable(False, False)
        self.root.attributes('-topmost', True)
        self.root.bind('<Escape>', lambda e: self.root.destroy())

    def update(self, text):
        self.label.config(text=text)
        self.root.update()

    def run(self):
        self.root.mainloop()

def check_pixel_color(main_img):
    h, w = main_img.shape[:2]
    for dx in range(15):
        x = pixel_coords[0] + dx
        y = pixel_coords[1]
        if not (0 <= x < w and 0 <= y < h):
            continue
        pixel_bgr = main_img[y, x]
        if np.all(pixel_bgr == expected_bgr):
            return True
    return False


def check_pixel_color_flush(main_img):
    h, w = main_img.shape[:2]
    if 0 <= flush_coords[0] < w and 0 <= flush_coords[1] < h:
        flush_bgr = main_img[flush_coords[1], flush_coords[0]]
        return np.all(flush_bgr == flush_expect_bgr)
    return False

def click_in_window(decision, window_left, window_top):
    if decision == "Bet":
        x, y = 920, 740
    elif decision == "Fold":
        x, y = 726, 740
    else:
        return
    # Kliknij w zadanym miejscu względem lewego górnego rogu okna Chrome
    pyautogui.click(window_left + x, window_top + y)

def background_task(window: ResultWindow):
    while True:
        screenshot, win_left, win_top = None, None, None
        while screenshot is None:
            windows = gw.getWindowsWithTitle(window_title)
            if windows and any(w.isActive for w in windows):
                screenshot, win_left, win_top = get_window_screenshot(window_title)
            if screenshot is None:
                window.update("Czekam na aktywne okno...")
                time.sleep(2)
        while True:
            windows = gw.getWindowsWithTitle(window_title)
            if not windows or not any(w.isActive for w in windows):
                break
            screenshot, win_left, win_top = get_window_screenshot(window_title)
            if screenshot is not None:
                if check_pixel_color(screenshot):
                    window.update("Pixel")
                    if (check_pixel_color_flush(screenshot)):
                        window.update(f"Decyzja: Bet")
                        click_in_window("Bet", win_left, win_top)
                    else:
                        results = recognize_cards(screenshot)
                        if all(card_number is not None and score > 0.65 for _, card_number, score in results):
                            card_numbers = [card_number for _, card_number, _ in results]
                            card_numbers_sorted = sorted(card_numbers)
                            decyzja = calculate_decision(card_numbers_sorted)
                            window.update(f"Decyzja: {decyzja}")
                            # Kliknięcie
                            click_in_window(decyzja, win_left, win_top)
                        else:
                            window.update("...")
                else:
                    window.update("No pixel")
            else:
                window.update("...")
            time.sleep(0.5)

if __name__ == '__main__':
    window = ResultWindow()
    threading.Thread(target=background_task, args=(window,), daemon=True).start()
    window.run()


Exception in thread Thread-4 (background_task):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.13_3.13.1520.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1043, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "d:\Python\pokerbot\venv\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.13_3.13.1520.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 994, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_28784\322201445.py", line 171, in background_task
    window.update("No pixel")
    ~~~~~~~~~~~~~^^^^^^^^^^^^
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_28784\322201445.py", line 102, in update
    self.label.config(text=text)
    ~~~~~~~~~~~~~~~~~^^^^^^^^^^^